In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE        

In [4]:
    pair = "EUR_USD"
    df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)

In [5]:
df_an = MACD(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [6]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0:
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0:
        return SELL
    return NONE

In [7]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

In [8]:
df_an.direction.value_counts()

 0    40452
-1     1752
 1     1752
Name: direction, dtype: int64

In [9]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100,macd_delta,macd_delta_prev,direction
0,2016-01-08 01:00:00+00:00,5607,1.09180,1.09230,1.08717,1.08844,1.09171,1.09214,1.08707,1.08833,...,1.09246,1.08727,1.08854,0.003176,0.003140,0.000036,1.081957,0.000036,NaN,0
1,2016-01-08 02:00:00+00:00,1261,1.08848,1.08906,1.08760,1.08872,1.08838,1.08898,1.08752,1.08863,...,1.08915,1.08769,1.08880,0.002873,0.003086,-0.000214,1.082112,-0.000214,0.000036,-1
2,2016-01-08 03:00:00+00:00,604,1.08870,1.08894,1.08800,1.08842,1.08861,1.08885,1.08792,1.08833,...,1.08902,1.08809,1.08850,0.002578,0.002985,-0.000406,1.082255,-0.000406,-0.000214,0
3,2016-01-08 04:00:00+00:00,543,1.08844,1.08844,1.08722,1.08740,1.08836,1.08836,1.08713,1.08733,...,1.08851,1.08731,1.08748,0.002237,0.002835,-0.000598,1.082372,-0.000598,-0.000406,0
4,2016-01-08 05:00:00+00:00,851,1.08743,1.08853,1.08734,1.08782,1.08734,1.08844,1.08725,1.08774,...,1.08862,1.08742,1.08790,0.001977,0.002664,-0.000686,1.082495,-0.000686,-0.000598,0


In [10]:
def run_pair(pair, time_d=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = MACD(df_an)
    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
    
    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross, axis=1)
    
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'direction', 'EMA_100' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d=time_d
    )
    
    gt.run_test()
    return gt.df_results
    

In [11]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=1)))

In [12]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -376.0
EUR_USD -15.0
GBP_JPY 74.5
USD_CAD -57.0


In [ ]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())